In [ ]:
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl


# 1) Construction du système flou

distance = ctrl.Antecedent(np.arange(0, 30.01, 0.05), 'Distance_to_next_car')
distance['v_small'] = fuzz.trapmf(distance.universe, [0, 0, 3, 7])
distance['small']   = fuzz.trimf(distance.universe, [3, 7, 12])
distance['perfect'] = fuzz.trimf(distance.universe, [10, 15, 20])
distance['big']     = fuzz.trimf(distance.universe, [18, 23, 27])
distance['v_big']   = fuzz.trapmf(distance.universe, [25, 28, 30, 30])

speed = ctrl.Antecedent(np.arange(-20, 20.01, 0.05), 'Speed_change')
speed['declining'] = fuzz.trimf(speed.universe, [-20, -10, -2])
speed['constant']  = fuzz.trapmf(speed.universe, [-5, -1, 1, 5])
speed['growing']   = fuzz.trimf(speed.universe, [2, 10, 20])

acc = ctrl.Consequent(np.arange(-3, 3.01, 0.01), 'Acceleration_Adjust')
acc['neg_big']   = fuzz.trapmf(acc.universe, [-3, -3, -2, -1])
acc['neg_small'] = fuzz.trimf(acc.universe, [-2, -1, 0])
acc['zero']      = fuzz.trimf(acc.universe, [-0.5, 0, 0.5])
acc['pos_small'] = fuzz.trimf(acc.universe, [0, 1, 2])
acc['pos_big']   = fuzz.trapmf(acc.universe, [1, 2, 3, 3])

rules = [
    ctrl.Rule(distance['v_small'] & speed['declining'], acc['neg_small']),
    ctrl.Rule(distance['v_small'] & speed['constant'],  acc['neg_big']),
    ctrl.Rule(distance['v_small'] & speed['growing'],   acc['neg_big']),
    ctrl.Rule(distance['small'] & speed['declining'], acc['zero']),
    ctrl.Rule(distance['small'] & speed['constant'],  acc['neg_small']),
    ctrl.Rule(distance['small'] & speed['growing'],   acc['neg_big']),
    ctrl.Rule(distance['perfect'] & speed['declining'], acc['pos_small']),
    ctrl.Rule(distance['perfect'] & speed['constant'],  acc['zero']),
    ctrl.Rule(distance['perfect'] & speed['growing'],   acc['neg_small']),
    ctrl.Rule(distance['big'] & speed['declining'], acc['pos_big']),
    ctrl.Rule(distance['big'] & speed['constant'],  acc['pos_small']),
    ctrl.Rule(distance['big'] & speed['growing'],   acc['zero']),
    ctrl.Rule(distance['v_big'] & speed['declining'], acc['pos_big']),
    ctrl.Rule(distance['v_big'] & speed['growing'],   acc['pos_small']),
]
system = ctrl.ControlSystem(rules)


In [23]:
sim = ctrl.ControlSystemSimulation(system)

def fuzzy_controller_reuse(sim, d, dv):
    sim.reset()
    sim.input['Distance_to_next_car'] = float(d)
    sim.input['Speed_change'] = float(dv)
    sim.compute()
    if not sim.output:
        return np.nan  
    return float(sim.output.get('Acceleration_Adjust', np.nan))


In [24]:
N = 1000
rng = np.random.default_rng(42)
X_distance = rng.uniform(0, 30, size=N)
X_speedchg = rng.uniform(-20, 20, size=N)

y = np.empty(N)
for i, (d, dv) in enumerate(zip(X_distance, X_speedchg)):
    y[i] = fuzzy_controller_reuse(sim, d, dv)

# suppression des éventuels NaN
mask = ~np.isnan(y)
X = np.column_stack([X_distance[mask], X_speedchg[mask]])
y = y[mask]


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor


# 4) Entraînement du modèle ML
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


# 5) Évaluation du modèle

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("\n--- Évaluation du modèle ML ---")
print("Erreur absolue moyenne (MAE) :", round(mae, 4))
print("Erreur quadratique moyenne (MSE) :", round(mse, 4))




--- Évaluation du modèle ML ---
Erreur absolue moyenne (MAE) : 0.0238
Erreur quadratique moyenne (MSE) : 0.0025


#### 4) Comparaison 
Logique floue :
 • Chaque règle utilisée peut être comprise par un humain (ex. « Si la distance est petite et la vitesse diminue, alors on réduit l’accélération »). 
 • Pas besoin de données d’entraînement.  
 • Résultats interprétables (chaque règle a un sens).  

Machine Learning :
 • Nécessite un dataset d’exemples.  
 • Peut apprendre automatiquement des relations complexes.  
 • Meilleure précision possible si beaucoup de données.  

#### 5) avantages offre le ML
- Haute précision numérique : il apprend automatiquement les relations optimales à partir des données, même non linéaires.  
- Capacité d’adaptation : plus il reçoit de données, plus il devient performant.  
- Généralisation : il peut s’adapter à des situations non explicitement couvertes par les règles floues.   